In [55]:
import torch
import torchvision.models as models
from dataset import *

In [56]:
class Args:
    def __init__(self):
        self.batch_size = 4
        self.gpu = 0

args = Args()
device = torch.device('cuda:%d' % args.gpu)
args.device = device

In [49]:
names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [69]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
model.eval()

train_dataset = YfccDataset(train=True, short=True)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4)

In [ ]:
new_label = torch.zeros((len(train_dataset), 2), device=device)
global_img_idx = 0
start_time = time.time()

for b_idx, data in enumerate(train_loader):
    bx, by = data
    bx = bx.permute(0, 3, 1, 2).to(device)
    by = by.permute(0, 3, 1, 2).to(device)                   
    with torch.no_grad():
        prediction = model(bx)
        for img_idx in range(args.batch_size):
            label = prediction[img_idx]['labels']
            box = prediction[img_idx]['boxes'][label == 1]
            score = prediction[img_idx]['scores'][label == 1]

            box = box[score > 0.9]
            score = score[score > 0.9]
            center_x = (box[:, 0] + box[:, 2]) / 2
            center_y = (box[:, 1] + box[:, 3]) / 2

            total_score = score.sum()
            avg_location = box.mean()

            new_label[global_img_idx, 1] = total_score
            new_label[global_img_idx, 0] = avg_location
            global_img_idx += 1
#         plt.imshow(bx[box_id].permute(1, 2, 0).cpu())
#         print(score)
#         plt.scatter(center_x.cpu(), center_y.cpu())
#         plt.show()
    if b_idx % 100 == 0:
        print("Iteration %d, time %.2f" % (b_idx, time.time() - start_time))

Iteration 0, time 0.94
Iteration 100, time 51.98
Iteration 200, time 103.16
Iteration 300, time 154.46
Iteration 400, time 205.82
Iteration 500, time 257.39
Iteration 600, time 308.80
Iteration 700, time 360.09
Iteration 800, time 411.39
Iteration 900, time 462.80
Iteration 1000, time 514.09
Iteration 1100, time 565.30


In [ ]:
torch.save(new_label, '/efs/yfcc/processed/label.pt')